In [1]:
from fsspec_reference_maker.hdf import SingleHdf5ToZarr 
from fsspec_reference_maker.combine import MultiZarrToZarr

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import zipfile
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob
import os

from azure.storage.blob import ContainerClient
import tempfile
from metpy import xarray

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [3]:
# from dask.distributed import Client
# client = Client()
# client

In [4]:
%load_ext snakeviz
import cProfile

In [5]:
%%prun -D profile2
fs = fsspec.filesystem('reference', fo='combined.json', remote_protocol='az', remote_options={'account_name':'goeseuwest'}, simple_templates=True)

 
*** Profile stats marshalled to file 'profile2'. 


         50685319 function calls (50684578 primitive calls) in 30.766 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    5.370    5.370    9.642    9.642 reference.py:265(_process_references1)
        1    4.334    4.334    4.334    4.334 {built-in method ujson.loads}
  2432934    4.157    0.000    6.740    0.000 spec.py:178(_strip_protocol)
        1    4.094    4.094   16.459   16.459 reference.py:347(_dircache_from_items)
  2432934    3.216    0.000   11.299    0.000 spec.py:898(_parent)
 13178616    1.520    0.000    1.520    0.000 {built-in method builtins.isinstance}
  1011538    1.507    0.000    1.507    0.000 {built-in method binascii.a2b_base64}
  5886297    1.107    0.000    1.107    0.000 {method 'startswith' of 'str' objects}
  2432613    0.683    0.000    0.683    0.000 {method 'rsplit' of 'str' objects}
  2432939    0.653    0.000    0.877    0.000 utils.py:284(stringify_path)
  4867068    0.595    

In [4]:
m = fs.get_mapper("")

In [5]:
%%time
ds = xr.open_dataset(m, engine='zarr')

CPU times: user 14.2 s, sys: 209 ms, total: 14.4 s
Wall time: 14.6 s


### Calculate true color
https://unidata.github.io/python-gallery/examples/mapping_GOES16_TrueColor.html

In [7]:
import IPython

In [9]:
%matplotlib notebook

nt = len(ds.t)

from matplotlib.animation import FuncAnimation

fig = plt.figure(figsize=(7.5,7.5), dpi=100)

dummy_channel = ds.metpy.parse_cf('CMI_C01')
x = dummy_channel.x; y = dummy_channel.y
ax = fig.add_subplot(111, projection = dummy_channel.metpy.cartopy_crs)

r = ds.isel(t=0)['CMI_C02']; r = np.clip(r, 0, 1)
g = ds.isel(t=0)['CMI_C03']; g = np.clip(g, 0, 1)
b = ds.isel(t=0)['CMI_C01']; b = np.clip(b, 0, 1)

gamma = 2.5; r = np.power(r, 1/gamma); g = np.power(g, 1/gamma); b = np.power(b, 1/gamma)
g_true = 0.45 * r + 0.1 * g + 0.45 * b
g_true = np.clip(g_true, 0, 1)
rgb_true = np.dstack((r, g_true, b))

p = ax.imshow(rgb_true, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()))
ax.coastlines()
ax.add_feature(ccrs.cartopy.feature.BORDERS)

def update_anim(i):
    r = ds.isel(t=i)['CMI_C02']; r = np.clip(r, 0, 1)
    g = ds.isel(t=i)['CMI_C03']; g = np.clip(g, 0, 1)
    b = ds.isel(t=i)['CMI_C01']; b = np.clip(b, 0, 1)

    gamma = 2.5; r = np.power(r, 1/gamma); g = np.power(g, 1/gamma); b = np.power(b, 1/gamma)
    g_true = 0.45 * r + 0.1 * g + 0.45 * b
    g_true = np.clip(g_true, 0, 1)
    rgb_true = np.dstack((r, g_true, b))
    
    p = ax.imshow(rgb_true, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()))
    ax.coastlines()
    ax.add_feature(ccrs.cartopy.feature.BORDERS)
    return p
  
# update_anim(1)
animator = FuncAnimation(fig, update_anim, frames=nt, interval=1)
plt.show()

<IPython.core.display.Javascript object>

/srv/conda/envs/notebook/lib/python3.8/site-packages/matplotlib/animation.py:973: UserWarning: Animation was deleted without rendering anything. This is most likely unintended. To prevent deletion, assign the Animation to a variable that exists for as long as you need the Animation.
  warnings.warn(


***
| Action | Time | Notes|
|  ----  |  --- |  --- |
| Open with fsspec | 23min 52s |  |
| Create Dataset (xarray) | 17.9s |  |

In [10]:
from IPython.display import HTML

NameError: name 'HTML' is not defined